#### Importando as bibliotecas

In [38]:
#!pip install boto3

In [46]:
import pandas as pd
import numpy as np
from io import BytesIO
from io import StringIO 
import boto3

#### Cliente para conectar o MinIO

In [47]:
client = boto3.client('s3', 
    endpoint_url='http://awari-minio-nginx:9000',
    aws_access_key_id="nN4wl0PlQAGGgFqG",
    aws_secret_access_key='AZj1hJDxu4HWflRZ1RqQl2x2UC23g25D',
    aws_session_token=None,
    config=boto3.session.Config(signature_version='s3v4'),
    verify=False,
    region_name='sa-east-1'
)

#### Lendo os arquivos

In [126]:
df_estados = pd.read_csv("dados/estados.csv") 
df_municipios = pd.read_csv("dados/municipios.csv")

In [127]:
df_estados.head()

,codigo_uf,uf,nome,latitude,longitude,regiao
0,11,RO,Rondônia,-10.83,-63.34,Norte
1,12,AC,Acre,-8.77,-70.55,Norte
2,13,AM,Amazonas,-3.47,-65.10,Norte
3,14,RR,Roraima,1.99,-61.33,Norte
4,15,PA,Pará,-3.79,-52.48,Norte


In [128]:
df_municipios.head()

,codigo_ibge,nome,latitude,longitude,capital,codigo_uf,siafi_id,ddd,fuso_horario
0,5200050,Abadia de Goiás,-16.75730,-49.4412,0,52,1050,62,America/Sao_Paulo
1,3100104,Abadia dos Dourados,-18.48310,-47.3916,0,31,4001,34,America/Sao_Paulo
2,5200100,Abadiânia,-16.19700,-48.7057,0,52,9201,62,America/Sao_Paulo
3,3100203,Abaeté,-19.15510,-45.4444,0,31,4003,37,America/Sao_Paulo
4,1500107,Abaetetuba,-1.72183,-48.8788,0,15,401,91,America/Sao_Paulo


### Atividade - 01

* Criar uma pasta nomeada com a sigla da UF para cada estado encontrado no arquivo JSON

In [129]:
lista_estados = df_estados['uf'].unique()
lista_estados.sort()

In [130]:
# Criando as pastas para cada Estado
for estado in lista_estados:
    client.put_object(Bucket='aula-06', Key=f'{estado}/')

### Atividade - 02

* Organizar as cidades por estado um único arquivo CSV, nomeado como cidades.csv;
* Salvar esse arquivo cidades.csv dentro da pasta da UF (estado) correspondente

* Dicionário contendo código e nome de cada municipio

In [131]:
dicionario = dict()
for i in df_estados[['uf', 'codigo_uf']].iterrows():
    cod = (i[1][1])
    uf = (i[1][0])
    dicionario[uf] = cod

* Enviando para as pastas os arquivos contendo informações do municipio do Estado.

In [137]:
for i in lista_estados:
    dff = df_municipios[df_municipios["codigo_uf"] == dicionario[i]]
    
    csv_buffer = StringIO()
    dff.to_csv(csv_buffer, index=False)
    client.put_object(Body=csv_buffer.getvalue(), Bucket='aula-06', Key=f"{i}/cidades.csv")

#### Verificando se os arquivos foram criandos corretamente

* Carregando o arquivo com as informações do Ceará

In [149]:
obj = client.get_object(
    Bucket='aula-06', 
    Key="CE/cidades.csv"
).get("Body")

data = pd.read_csv(obj)
data.head()

,codigo_ibge,nome,latitude,longitude,capital,codigo_uf,siafi_id,ddd,fuso_horario
0,2300101,Abaiara,-7.34588,-39.0416,0,23,1301,88,America/Sao_Paulo
1,2300150,Acarape,-4.22083,-38.7055,0,23,1231,85,America/Sao_Paulo
2,2300200,Acaraú,-2.88769,-40.1183,0,23,1303,88,America/Sao_Paulo
3,2300309,Acopiara,-6.08911,-39.4480,0,23,1305,88,America/Sao_Paulo
4,2300408,Aiuaba,-6.57122,-40.1178,0,23,1307,88,America/Sao_Paulo


* Extra - Listando nomes dos arquivos presentes em um buket

In [177]:
url_arquivos = []

for uf in lista_estados:
    bucket_name = 'aula-06'
    folder_prefix = uf

    response = client.list_objects_v2(
        Bucket=bucket_name,
        Prefix=folder_prefix
    )
    
    url_arquivos.append(response['Contents'][1]["Key"])

* Lendo cada arquivo presente em um buket

In [178]:
for url in url_arquivos:

    obj = client.get_object(
        Bucket='aula-06', 
        Key=url
    ).get("Body")

    data = pd.read_csv(obj)
    break

In [181]:
data.head()

,codigo_ibge,nome,latitude,longitude,capital,codigo_uf,siafi_id,ddd,fuso_horario
0,1200013,Acrelândia,-9.82581,-66.8972,0,12,643,68,America/Rio_Branco
1,1200054,Assis Brasil,-10.92980,-69.5738,0,12,157,68,America/Rio_Branco
2,1200104,Brasiléia,-10.99500,-68.7497,0,12,105,68,America/Rio_Branco
3,1200138,Bujari,-9.81528,-67.9550,0,12,645,68,America/Rio_Branco
4,1200179,Capixaba,-10.56600,-67.6860,0,12,647,68,America/Rio_Branco


### Atividade - 03
* Exportar os dados para o MongoDB e visualizá-los a partir do banco de dados.